In [21]:
import pandas as pd
import requests
import json
import datetime as dt

https://open-meteo.com/en/docs/historical-weather-api#start_date=2021-07-01&end_date=2023-10-17&hourly=temperature_2m,relativehumidity_2m,precipitation,weathercode,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,is_day,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&timezone=America%2FSao_Paulo

In [43]:
# response = requests.get(
#   'https://archive-api.open-meteo.com/v1/archive?latitude=-22.908351&longitude=-53.112785&start_date=2021-07-16&end_date=2023-10-17&hourly=temperature_2m,relativehumidity_2m,precipitation,weathercode,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,is_day,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&timezone=America%2FSao_Paulo')

In [44]:
# json_data = response.json()

In [62]:
# with open("loanda_weather.json", "w") as outfile:
#     json.dump(json_data, outfile)

In [5]:
# Opening JSON file
f = open('loanda_weather.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

In [97]:
# Cleaning hour_data
hour_data = pd.DataFrame.from_dict(data['hourly'])
columns_with_units = []
for key, units in data['hourly_units'].items():
    columns_with_units.append(key + '_in_'+units)
hour_data.columns = columns_with_units
hour_data['time_in_iso8601'] = pd.to_datetime(hour_data['time_in_iso8601'])
hour_data['time_day_for_join'] = hour_data['time_in_iso8601'].dt.date.astype(str)
hour_data = hour_data.dropna()

# Cleaning daily_data
day_data = pd.DataFrame.from_dict(data['daily'])
columns_with_units = []
for key, units in data['daily_units'].items():
    columns_with_units.append(key + '_in_'+units)
day_data.columns = columns_with_units
day_data['time_in_iso8601'] = pd.to_datetime(day_data['time_in_iso8601'])
day_data['time_day_for_join'] = day_data['time_in_iso8601'].astype(str)
del day_data['time_in_iso8601']

# Merge All
data_all = hour_data.merge(day_data, on = 'time_day_for_join', how = 'left')

# Tratamentos:

# Adicionar tempo de luz do dia com sunrise e sunset

data_all['sunrise_in_iso8601'] = pd.to_datetime(data_all['sunrise_in_iso8601'])
data_all['sunset_in_iso8601'] = pd.to_datetime(data_all['sunset_in_iso8601'])
time_diff = (data_all['sunset_in_iso8601'] - data_all['sunrise_in_iso8601'] )
data_all['time_of_sunlight_in_iso8601'] = (time_diff.dt.days * 24) + (time_diff.dt.seconds / 3600)

In [110]:
data_all.astype(str).to_json('loanda_weather_curated.json')

In [111]:
# Opening JSON file Curated
f = open('loanda_weather_curated.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

In [112]:
pd.DataFrame.from_dict(data)

,time_in_iso8601,temperature_2m_in_°C,relativehumidity_2m_in_%,precipitation_in_mm,weathercode_in_wmo code_x,cloudcover_in_%,cloudcover_low_in_%,cloudcover_mid_in_%,cloudcover_high_in_%,windspeed_10m_in_km/h,...,sunset_in_iso8601,precipitation_sum_in_mm,rain_sum_in_mm,precipitation_hours_in_h,windspeed_10m_max_in_km/h,windgusts_10m_max_in_km/h,winddirection_10m_dominant_in_°,shortwave_radiation_sum_in_MJ/m²,et0_fao_evapotranspiration_in_mm,time_of_sunlight_in_iso8601
0,2021-07-16 00:00:00,20.4,68.0,0.3,51.0,73.0,0.0,100.0,42.0,9.4,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
1,2021-07-16 01:00:00,19.1,79.0,0.8,53.0,70.0,0.0,100.0,35.0,9.0,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
2,2021-07-16 02:00:00,18.0,89.0,1.1,55.0,76.0,2.0,100.0,46.0,10.3,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
3,2021-07-16 03:00:00,17.4,94.0,1.4,61.0,80.0,8.0,100.0,44.0,10.7,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
4,2021-07-16 04:00:00,17.2,95.0,1.1,55.0,74.0,10.0,99.0,17.0,10.0,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19624,2023-10-11 16:00:00,34.5,42.0,0.0,0.0,15.0,0.0,14.0,22.0,12.3,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
19625,2023-10-11 17:00:00,34.0,42.0,0.0,0.0,16.0,0.0,20.0,12.0,11.3,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
19626,2023-10-11 18:00:00,32.5,48.0,0.0,1.0,27.0,0.0,23.0,43.0,7.5,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
19627,2023-10-11 19:00:00,30.9,51.0,0.0,1.0,40.0,0.0,28.0,76.0,8.7,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
